In [1]:
import os
import csv
import base64
import openai
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
from prepare import convert_ipynb_to_pdf,convert_pdf_to_base64_images,convert_docx_to_base64_images,convert_pptx_to_base64_images
load_dotenv()
HOMEWORK_DIR = os.path.abspath(os.getenv('HOMEWORK_DIR'))
# 確認路徑是否存在
if not os.path.exists(HOMEWORK_DIR):
    raise FileNotFoundError(f"The directory {HOMEWORK_DIR} does not exist. Current path: {HOMEWORK_DIR}")
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
with open('assignment_requirements.txt', 'r', encoding='utf-8') as f:
    assignment_requirements = f.read()

In [3]:
# 讀取學生檔案並分類
def read_files(student):
    image_base64_list = []  # 用於存放Base64格式的圖片
    text_contents = []  # 用於存放文本內容或錯誤訊息
    
    for text_file in student['texts']:
        text_path = os.path.join(student['path'], text_file)
        
        # texts 型態檔案處理
        if text_file.lower().endswith(('.py', '.java', '.cpp', '.txt')):
            try:
                with open(text_path, 'r', encoding='utf-8') as file_content:
                    text_contents.append(file_content.read())
            except Exception as e:
                print(f"檔案無法讀取: {text_path,str(e)}")
    
    # 處理直接放在 images 資料夾裡的圖片 (.png, .jpg, .jpeg)
    for image_file in student['images']:
        image_path = os.path.join(student['path'], image_file)
        if image_file.lower().endswith('.ipynb'):
            pdf_path = convert_ipynb_to_pdf(image_file)
            if "無法讀取的檔案" in pdf_path:
                text_contents.append(pdf_path)  # 添加錯誤信息
            else:
                base64_images = convert_pdf_to_base64_images(image_path)
                image_base64_list.extend(base64_images)  # 加入Base64圖片列表
        
        elif image_file.lower().endswith('.docx'):
            base64_images = convert_docx_to_base64_images(image_path)
            image_base64_list.extend(base64_images)  # 加入Base64圖片列表
        
        elif image_file.lower().endswith('.pptx'):
            base64_images = convert_pptx_to_base64_images(image_path)
            image_base64_list.extend(base64_images)  # 加入Base64圖片列表
        elif image_file.lower().endswith('.pdf'):
            base64_images = convert_pdf_to_base64_images(image_path)
            image_base64_list.extend(base64_images)  # 加入Base64圖片列表
            
        else:
            try:
                with Image.open(image_path) as img:
                    buffered = BytesIO()
                    img.save(buffered, format="PNG")
                    img_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
                    image_base64_list.append(img_base64)
            except Exception as e:
                print(f"圖片檔案無法讀取: {image_path,str(e)}")


    return image_base64_list, text_contents


In [4]:
SystemPrompt = '''
As an impartial and neutral educator, 
your objective is to evaluate students' submitted assignments according to the specified grading criteria. 
For each student, please assign a score out of 100 based on the quality of their work and adherence to the assignment requirements. 
Additionally, provide a brief comment in Traditional Chinese that reflects your assessment and is limited to 10 characters. 
Ensure that your output follows this structured format: "Student ID, score, comment." 
Refrain from including any additional text or information
'''


In [5]:
from langchain_openai import ChatOpenAI

# 定義評分函數，根據檔案類型進行不同處理
def grade_assignment(student, image_base64_list, text_contents):
    messages = [
        {"role": "system", "content": SystemPrompt}
    ]
    
    # 添加作業要求
    messages.append({"role": "user", "content": f"作業內容：\n{assignment_requirements}"})
    
    # 添加學生資訊
    messages.append({"role": "user", "content": f"學生資訊：{student['id']} - {student['name']}"})
    
    # 添加 images 型態的圖片（作業內容）
    for image_base64 in image_base64_list:
        if "無法轉換的檔案" in image_base64:
            messages.append({"role": "user", "content": image_base64})
        else:
            messages.append({
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{image_base64}",
                    }
                },
            ],
        })
    
    # 添加 texts 型態的文本內容（程式碼或錯誤信息）
    for content in text_contents:
        messages.append({"role": "user", "content": content})
    # 調用 OpenAI API
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0,max_tokens=20)
    response = llm.invoke(messages)
    text = response.content
    # 解析回應
    lines = text.strip().split(',')
    score = 0
    comment = ''
    studentID = lines[0]
    score = lines[1]
    comment = lines[2]
    return studentID, score, comment


In [6]:
# 遍歷學生資料夾
students = []
step=0
for student_folder in os.listdir(HOMEWORK_DIR):
    student_path = os.path.join(HOMEWORK_DIR, student_folder)
    if os.path.isdir(student_path):
        try:
            student_id, student_name = student_folder.split('_', 1)
        except ValueError:
            continue
        # 如果資料夾名稱以 "無附件" 結尾
        if student_folder.endswith('無附件'):
            students.append({
                'id': student_id,
                'name': student_name,
                'images': [],
                'texts': [],
                'path': student_path,
                'has_attachments': False
            })
        else:
            files = os.listdir(student_path)
            images = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg','.pdf','.ipynb','.docx', '.pptx'))]
            texts = [f for f in files if f.lower().endswith(('.py', '.java', '.cpp',  '.txt',))]
            students.append({
                'id': student_id,
                'name': student_name,
                'images': images,
                'texts': texts,
                'path': student_path,
                'has_attachments': True
            })

In [7]:
# 開始評分
with open('grading_results.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['學號', '姓名', '分數', '評語']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for student in students:
        if not student['has_attachments']:
            # 若無附件，設定分數為 0 並給予評語
            writer.writerow({'學號': student['id'], '姓名': student['name'], '分數': 0, '評語': '未繳交'})
            print({'學號': student['id'], '姓名': student['name'], '分數': 0, '評語': '未繳交'})
        else:
            image_base64_list, text_contents = read_files(student)
            if image_base64_list or text_contents:
                studentID,score, comment = grade_assignment(student, image_base64_list, text_contents)
                print({'學號': student['id'], '姓名': student['name'], '分數': score, '評語': comment})
            else:
                score = 100
                comment = '讀取異常'
                print({'學號': student['id'], '姓名': student['name'], '分數': score, '評語': '讀取異常'})
            writer.writerow({'學號': student['id'], '姓名': student['name'], '分數': score, '評語': comment})

{'學號': '412777343', '姓名': '張仕敬', '分數': ' 85', '評語': ' 表達清晰。'}
{'學號': '411770026', '姓名': '李佩芸', '分數': ' 85', '評語': ' 表達清晰。'}
